# Graphs generation

In [1]:
from sampo.generator.base import SimpleSynthetic

In [2]:
GRAPHS_TOP_BORDER = 100
GRAPHS_COUNT = 10

In [7]:
ss = SimpleSynthetic(111)

graphs = [ss.work_graph(top_border=GRAPHS_TOP_BORDER) for _ in range(GRAPHS_COUNT)]

# Metrics calculation

In [8]:
from sampo.schemas.graph import WorkGraph


def metric_vertex_count(wg: WorkGraph) -> float:
    return wg.vertex_count

def metric_min_children(wg: WorkGraph) -> float:
    return min((len(node.children) for node in wg.nodes if node.children))

def metric_max_children(wg: WorkGraph) -> float:
    return max((len(node.children) for node in wg.nodes if node.children))

def metric_min_parents(wg: WorkGraph) -> float:
    return min((len(node.parents) for node in wg.nodes if node.parents))

def metric_max_parents(wg: WorkGraph) -> float:
    return max((len(node.parents) for node in wg.nodes if node.parents))

In [9]:
dataset = [(
    metric_vertex_count(wg),
    metric_min_children(wg),
    metric_max_children(wg),
    metric_min_parents(wg),
    metric_max_parents(wg)
) for wg in graphs]

dataset

[(46, 1, 31, 1, 32),
 (150, 1, 60, 1, 42),
 (129, 1, 42, 1, 48),
 (131, 1, 40, 1, 52),
 (198, 1, 64, 1, 40),
 (129, 1, 42, 1, 58),
 (139, 1, 46, 1, 64),
 (41, 1, 27, 1, 27),
 (141, 1, 42, 1, 59),
 (135, 1, 43, 1, 58)]

In [14]:
from sampo.schemas.time import Time
from sampo.scheduler.topological.base import TopologicalScheduler
from sampo.scheduler.heft.base import HEFTScheduler, HEFTBetweenScheduler

contractors = [ss.contractor(10)]
schedulers = [HEFTScheduler(), HEFTBetweenScheduler(), TopologicalScheduler()]

def generate_result(i: int, wg: WorkGraph) -> list[float]:
    if i % 10 == 0:
        print(f'{i}/{len(graphs)} processed')
    return [int(scheduler.schedule(wg, contractors).execution_time) for scheduler in schedulers]

schedulers_results = [generate_result(i, wg) for i, wg in enumerate(graphs)]
schedulers_results

0/10 processed


[[434, 1507, 1691],
 [1399, 5005, 5666],
 [958, 3837, 4346],
 [1224, 4316, 4901],
 [1813, 6860, 7750],
 [1027, 3911, 4385],
 [1201, 4972, 5573],
 [415, 1298, 1496],
 [1277, 4768, 5352],
 [1213, 4364, 4904]]

In [15]:
def argmin(array) -> int:
    res = 0
    res_v = int(Time.inf())
    for i, v in enumerate(array):
        if v < res_v:
            res_v = v
            res = i
    return res

schedulers_classification = [argmin(result) for result in schedulers_results]
schedulers_classification

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]